In [17]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import pandas as pd
import numpy as np
import oandapyV20.endpoints.orders as orders
from Moduled_functions import tranformation
from Moduled_functions import calculate_ema_slope
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
bearish_model = load_model('/work/Trained_Models/FX_Bearish_model_2025-02-19.keras')
bullish_model = load_model('/work/Trained_Models/FX_Bullish_model_2025-02-21.keras')

2025-04-06 13:45:24.933481: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-06 13:45:24.936378: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-06 13:45:24.967382: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-06 13:45:24.967496: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-06 13:45:24.968499: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [28]:
access_token = '320c03e29b3723c3869a2eafa278c553-c96eccee26e4e5864dcf001e98f7db84'
account_id_demo = '101-004-31059296-001'

In [3]:
def place_order1(current_price,stoploss,target,qty,token,account_id):
    order_details = {
    "order": {
        #"price": current_price,
        #"stopLossOnFill": {
        "trailingStopLossOnFill" : {   
            "timeInForce": "GTC",
            "distance": stoploss
        },
        "takeProfitOnFill": {
            "timeInForce": "GTC",
            "price": target
        },
        "timeInForce": "FOK",
        "instrument": "EUR_USD",
        "units": qty,
        "type": "MARKET",
        "positionFill": "DEFAULT"
    }
    }

    client = oandapyV20.API(access_token=token)
    r = orders.OrderCreate(accountID=account_id, data=order_details)
    res = client.request(r)
    return res


In [4]:
from Moduled_functions import get_data
from Moduled_functions import place_order
base_data,current_price = get_data(100, access_token)

take_profit = current_price * 0.9985  # 0.15% above
stop_loss = current_price * 0.0005
qty = '-1'

In [30]:
place_order1(str(current_price),str(round(stop_loss,5)),str(round(take_profit,5)),qty,access_token,account_id_demo)

{'orderCreateTransaction': {'id': '969',
  'accountID': '101-004-31059296-001',
  'userID': 31059296,
  'batchID': '969',
  'requestID': '61357230905113915',
  'time': '2025-04-04T21:25:22.414997332Z',
  'type': 'MARKET_ORDER',
  'instrument': 'EUR_USD',
  'units': '-1',
  'timeInForce': 'FOK',
  'positionFill': 'DEFAULT',
  'takeProfitOnFill': {'price': '1.09468', 'timeInForce': 'GTC'},
  'trailingStopLossOnFill': {'distance': '0.00055',
   'timeInForce': 'GTC',
   'triggerMode': 'TOP_OF_BOOK'},
  'reason': 'CLIENT_ORDER'},
 'orderCancelTransaction': {'id': '970',
  'accountID': '101-004-31059296-001',
  'userID': 31059296,
  'batchID': '969',
  'requestID': '61357230905113915',
  'time': '2025-04-04T21:25:22.414997332Z',
  'type': 'ORDER_CANCEL',
  'orderID': '969',
  'reason': 'MARKET_HALTED'},
 'relatedTransactionIDs': ['969', '970'],
 'lastTransactionID': '970'}

In [3]:
    params = {
    "granularity": "M5",
    "from" : '2024-10-17'
    }
    client = oandapyV20.API(access_token=access_token)
    r = instruments.InstrumentsCandles(instrument="EUR_USD",params=params)
    dd = client.request(r)
    candles = pd.DataFrame.from_records([{
        'datetime': candle['time'],
        'Open': float(candle['mid']['o']),
        'High': float(candle['mid']['h']),
        'Low': float(candle['mid']['l']),
        'Close': float(candle['mid']['c']),
        'volume': candle['volume'],
    } for candle in dd['candles']])

    candles[['Open', 'High', 'Low', 'Close']] = candles[['Open', 'High', 'Low', 'Close']].apply(lambda x: x*10000)
    candles['datetime'] = pd.to_datetime(candles['datetime'])

In [365]:
def fetch_candles(from_date):
    params = {
        "granularity": "M5",
        "from": from_date,
        "count": 500
    }
    r = instruments.InstrumentsCandles(instrument="EUR_USD", params=params)
    dd = client.request(r)
    df = pd.DataFrame.from_records([{
        'datetime': candle['time'],
        'Open': float(candle['mid']['o']),
        'High': float(candle['mid']['h']),
        'Low': float(candle['mid']['l']),
        'Close': float(candle['mid']['c']),
        'volume': candle['volume'],
    } for candle in dd['candles']])
    
    df[['Open', 'High', 'Low', 'Close']] = df[['Open', 'High', 'Low', 'Close']].apply(lambda x: x*10000)
    df['datetime'] = pd.to_datetime(df['datetime'])
    return df

# Initialize variables
start_date = '2020-01-01'
consolidated_df = pd.DataFrame()
current_from = start_date
max_requests = 15000

for _ in range(max_requests):
    try:
        print(f"Fetching data from: {current_from}")
        new_data = fetch_candles(current_from)
        
        if new_data.empty:
            break
            
        consolidated_df = pd.concat([consolidated_df, new_data], ignore_index=True)
        
        # Get the maximum datetime and format it for next iteration
        max_datetime = new_data['datetime'].max()
        current_from = max_datetime.strftime('%Y-%m-%d')
        
        time.sleep(1)
        
    except Exception as e:
        print(f"Error: {e}")
        break

# Remove any duplicates and sort
consolidated_df = consolidated_df.drop_duplicates(subset='datetime').sort_values('datetime')
print(f"\nTotal rows in consolidated data: {len(consolidated_df)}")
consolidated_df

Fetching data from: 2020-06-11
Fetching data from: 2020-06-14
Fetching data from: 2020-06-16
Fetching data from: 2020-06-17
Fetching data from: 2020-06-18
Fetching data from: 2020-06-21
Fetching data from: 2020-06-23
Fetching data from: 2020-06-24
Fetching data from: 2020-06-25
Fetching data from: 2020-06-28
Fetching data from: 2020-06-30
Fetching data from: 2020-07-01
Fetching data from: 2020-07-02
Fetching data from: 2020-07-05
Fetching data from: 2020-07-07
Fetching data from: 2020-07-08
Fetching data from: 2020-07-09
Fetching data from: 2020-07-12
Fetching data from: 2020-07-14
Fetching data from: 2020-07-15
Fetching data from: 2020-07-16
Fetching data from: 2020-07-19
Fetching data from: 2020-07-21
Fetching data from: 2020-07-22
Fetching data from: 2020-07-23
Fetching data from: 2020-07-26
Fetching data from: 2020-07-28
Fetching data from: 2020-07-29
Fetching data from: 2020-07-30
Fetching data from: 2020-08-02
Fetching data from: 2020-08-04
Fetching data from: 2020-08-05
Fetching

KeyboardInterrupt: 

In [367]:
consolidated_df.to_csv('/work/Insights/Combined_data_volume_2020.csv')

In [13]:
consolidated_df= pd.read_csv('/work/Insights/Combined_data_volume_2020.csv')
consolidated_df['datetime'] = pd.to_datetime(consolidated_df['datetime'])
consolidated_df.drop(columns=['Unnamed: 0'],inplace=True)

In [19]:
# Transform the data
base_data = tranformation(consolidated_df, 0.0015)

# Calculate EMA slopes
base_data['EMA_slope'] = calculate_ema_slope(base_data, 'Open', 9)
base_data['EMA_slope_15'] = calculate_ema_slope(base_data, 'Open', 15)
base_data['EMA_slope_60'] = calculate_ema_slope(base_data, 'Open', 60)

# Calculate deviation
base_data['deviation_'] = base_data['bullish_move_flag_20'] + base_data['bearish_move_flag_20']
X = base_data.iloc[:,9:69]
y = base_data['deviation_']

# Normalize the data
scaler = StandardScaler()
scaler.fit(X)
X_normalized = pd.DataFrame(scaler.transform(X)).round(1)

# Get predictions
y_pred_bearish = bearish_model.predict(X_normalized, verbose=0)
y_pred_bearish = pd.DataFrame(y_pred_bearish)

y_pred_bullish = bullish_model.predict(X_normalized, verbose=0)
y_pred_bullish = pd.DataFrame(y_pred_bullish)

# Combine results
result_combined = pd.concat([y_pred_bearish, y_pred_bullish, y], axis=1)
result_combined.columns = ['bearish_0','bearish_1','bearish_2','bearish_3',
                            'bullish_0','bullish_1','bullish_2','bullish_3','y_true']

base_data = pd.concat([base_data, result_combined], axis=1)

In [22]:
Can you isolate the bearish movement where y_true =2 and other values combined together. Evaluate feature importance from the below list, 
with p-test validation confirm how good is the assumption here.
Also the ranges are required to confirm future assumptions for the top figured out feature.
   ,'move_from_bottom'
    ,'EMA_slope'
    ,'volume'
    ,'bearish_2'
    ,'bullish_1'
    ,'EMA_slope_60'
    ,'move_from_top'
    ,'volume_std_mean'
    ,'volume_mean_5'
    ,'volume_mean_3'

SyntaxError: invalid syntax (2766469465.py, line 1)

> Can you isolate the bearish movement where y_true =2 and other values combined together. Evaluate feature importance from the below list, 
with p-test validation confirm how good is the assumption here.
Also the ranges are required to confirm future assumptions for the top figured out feature.
   ,'move_from_bottom'
    ,'EMA_slope'
    ,'volume'
    ,'bearish_2'
    ,'bullish_1'
    ,'EMA_slope_60'
    ,'move_from_top'
    ,'volume_std_mean'
    ,'volume_mean_5'
    ,'volume_mean_3'

In [24]:
base_data['date'] = base_data['datetime'].dt.date
CY_base_data = base_data[base_data['date'] > pd.to_datetime('2025-01-01').date()]

In [ ]:
#original bearish indictor values
base_data['bearish_flag']  = base_data.apply(lambda x:2 if x['volume_mean_5'] >590 ##433 
                                                        and x['volume_mean_5'] <716
                                                        and x['bearish_2'] > 0.510 
                                                        and x['bearish_2'] < 0.626 
                                                        and x['bullish_1'] > 0.509 
                                                        and x['bullish_1'] < 0.609
                                                        and x['move_from_top'] > 0.089 
                                                        and x['move_from_top'] < 0.249
                                                        and x['volume'] > 450 ##410 
                                                        and x['volume'] <710
                                                        and x['volume_mean_3'] > 418
                                                        and x['volume_mean_3'] <732 else 0, axis=1)

### The optimal parameters are:
- Volume: 450-700
- Volume Mean 3: 500-750
- Volume Mean 5: 450-750
- Bearish_2: 0.50-0.65
- Move from Top: 0.08-0.25

In [50]:
CY_base_data['bearish_flag']  = CY_base_data.apply(lambda x:1 if x['volume_mean_5'] >450 ##433 
                                                        and x['volume_mean_5'] <750
                                                        and x['bearish_2'] > 0.50 
                                                        and x['bearish_2'] < 0.65 
                                                        #and x['bullish_1'] > 0.509 
                                                        #and x['bullish_1'] < 0.609
                                                        and x['move_from_top'] > 0.08
                                                        and x['move_from_top'] < 0.25
                                                        and x['volume'] > 450 ##410 
                                                        and x['volume'] <700
                                                        and x['volume_mean_3'] > 500
                                                        and x['volume_mean_3'] <750 else 0, axis=1)

/tmp/ipykernel_333/1479363735.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CY_base_data['bearish_flag']  = CY_base_data.apply(lambda x:1 if x['volume_mean_5'] >450 ##433


In [52]:
data = CY_base_data[CY_base_data['bearish_flag'] ==1]['y_true'].value_counts()
((data[2])*50000) + ((data[0]+data[1]+data[3])*-17000)

1398000

In [32]:
data

y_true
0    435
2    251
1    194
3     27
Name: count, dtype: int64

### Parameters:
volume: (450, 700)
volume_mean_3: (600, 700)
volume_mean_5: (500, 700)
bullish_1: (0.52, 0.6)
move_from_bottom: (0.12, 0.2)

In [34]:
## bullish indicator
CY_base_data['bullish_flag'] = CY_base_data.apply(lambda x:1 if x['volume'] >= 450
                                                        and x['volume'] <= 700
                                                        and x['volume_mean_3'] >= 600
                                                        and x['volume_mean_3'] <= 700
                                                        and x['volume_mean_5'] >= 500
                                                        and x['volume_mean_5'] <= 700
                                                        and x['bullish_1'] >= 0.52
                                                        and x['bullish_1'] <= 0.60
                                                        and x['move_from_bottom'] >= 0.12
                                                        and x['move_from_bottom'] <= 0.20 else 0, axis=1)

/tmp/ipykernel_333/3348931055.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CY_base_data['bullish_flag'] = CY_base_data.apply(lambda x:1 if x['volume'] >= 450


In [ ]:
#original bullish indictor values
base_data['bullish_flag'] = base_data.apply(lambda x:1 if x['volume'] >= 600##450
                                                    and x['volume'] <= 700
                                                    and x['volume_mean_3'] >= 550
                                                    and x['volume_mean_3'] <= 700
                                                    and x['volume_mean_5'] >= 590##550
                                                    and x['volume_mean_5'] <= 700
                                                    and x['bullish_1'] >= 0.51
                                                    and x['bullish_1'] <= 0.60
                                                    and x['move_from_bottom'] >= 0.10
                                                    and x['move_from_bottom'] <= 0.20 else 0, axis=1)

In [36]:
data = CY_base_data[CY_base_data['bullish_flag'] ==1]['y_true'].value_counts()
((data[1])*50000) + ((data[0]+data[2]+data[3])*-17000)

129000

In [38]:
data

y_true
0    31
2    28
1    24
3     4
Name: count, dtype: int64

In [58]:
(CY_base_data['date'].nunique()-12)

63

In [64]:
(len(CY_base_data[CY_base_data['bearish_flag']==1]) + len(CY_base_data[CY_base_data['bullish_flag']==1])) / (CY_base_data['date'].nunique()-12)

15.777777777777779

In [467]:
(len(base_data[base_data['bearish_flag']==2]) + len(base_data[base_data['bullish_flag']==1])) / base_data['date'].nunique()

9.33435207823961

> for the above bullish flag find a combination which has highest value profit booked from the features mentioned above.
Every correct bullish prediction, earns a USD 5000 profit and non-bullish move losses USD1700. Can you create a combination which has both higher profitability and good amount of trades also being made (not much tighter restrictions are preferred) 

In [439]:
# Let's create a function to calculate profit
def calculate_profit(group):
    profit = (group[group['y_true'] == 1].shape[0] * 50000) - \
             ((group[group['y_true'] != 1].shape[0]) * 17000)
    trades = len(group)
    return profit, trades

# Create ranges to test for each feature
feature_ranges = {
    'volume': [(450, 700), (500, 700), (550, 700), (600, 700)],
    'volume_mean_3': [(450, 700), (500, 700), (550, 700), (600, 700)],
    'volume_mean_5': [(450, 700), (500, 700), (550, 700), (600, 700)],
    'bullish_1': [(0.50, 0.60), (0.51, 0.60), (0.52, 0.60)],
    'move_from_bottom': [(0.08, 0.20), (0.10, 0.20), (0.12, 0.20)]
}

best_profit = float('-inf')
best_combo = None
best_trades = 0

# Test different combinations
for vol_range in feature_ranges['volume']:
    for vol3_range in feature_ranges['volume_mean_3']:
        for vol5_range in feature_ranges['volume_mean_5']:
            for bull_range in feature_ranges['bullish_1']:
                for move_range in feature_ranges['move_from_bottom']:
                    
                    # Apply conditions
                    filtered_data = base_data[
                        (base_data['volume'] >= vol_range[0]) &
                        (base_data['volume'] <= vol_range[1]) &
                        (base_data['volume_mean_3'] >= vol3_range[0]) &
                        (base_data['volume_mean_3'] <= vol3_range[1]) &
                        (base_data['volume_mean_5'] >= vol5_range[0]) &
                        (base_data['volume_mean_5'] <= vol5_range[1]) &
                        (base_data['bullish_1'] >= bull_range[0]) &
                        (base_data['bullish_1'] <= bull_range[1]) &
                        (base_data['move_from_bottom'] >= move_range[0]) &
                        (base_data['move_from_bottom'] <= move_range[1])
                    ]
                    
                    if len(filtered_data) > 0:
                        profit, trades = calculate_profit(filtered_data)
                        
                        # Update best combination if profit is higher and trades are reasonable
                        if profit > best_profit and trades >= 100:  # minimum 100 trades
                            best_profit = profit
                            best_combo = {
                                'volume': vol_range,
                                'volume_mean_3': vol3_range,
                                'volume_mean_5': vol5_range,
                                'bullish_1': bull_range,
                                'move_from_bottom': move_range
                            }
                            best_trades = trades

print("Best Combination Found:")
print(f"Profit: ${best_profit:,}")
print(f"Number of Trades: {best_trades}")
print("\nParameters:")
for feature, range_val in best_combo.items():
    print(f"{feature}: {range_val}")

# Let's analyze the distribution of y_true for this combination
best_filtered = base_data[
    (base_data['volume'] >= best_combo['volume'][0]) &
    (base_data['volume'] <= best_combo['volume'][1]) &
    (base_data['volume_mean_3'] >= best_combo['volume_mean_3'][0]) &
    (base_data['volume_mean_3'] <= best_combo['volume_mean_3'][1]) &
    (base_data['volume_mean_5'] >= best_combo['volume_mean_5'][0]) &
    (base_data['volume_mean_5'] <= best_combo['volume_mean_5'][1]) &
    (base_data['bullish_1'] >= best_combo['bullish_1'][0]) &
    (base_data['bullish_1'] <= best_combo['bullish_1'][1]) &
    (base_data['move_from_bottom'] >= best_combo['move_from_bottom'][0]) &
    (base_data['move_from_bottom'] <= best_combo['move_from_bottom'][1])
]

print("\nDistribution of outcomes:")
print(best_filtered['y_true'].value_counts())

Best Combination Found:
Profit: $3,398,000
Number of Trades: 1416

Parameters:
volume: (450, 700)
volume_mean_3: (600, 700)
volume_mean_5: (500, 700)
bullish_1: (0.52, 0.6)
move_from_bottom: (0.12, 0.2)

Distribution of outcomes:
y_true
0    512
2    452
1    410
3     42
Name: count, dtype: int64


In [441]:
# Calculate average trades per day with this strategy
trades_per_day = best_trades / base_data['date'].nunique()
success_rate = (best_filtered[best_filtered['y_true'] == 1].shape[0] / best_trades) * 100

print(f"Average trades per day: {trades_per_day:.2f}")
print(f"Success rate: {success_rate:.2f}%")
print(f"Average profit per trade: ${best_profit/best_trades:,.2f}")

# Update the bullish flag with the best combination
base_data['bullish_flag'] = base_data.apply(
    lambda x: 1 if (x['volume'] >= best_combo['volume'][0]
                   and x['volume'] <= best_combo['volume'][1]
                   and x['volume_mean_3'] >= best_combo['volume_mean_3'][0]
                   and x['volume_mean_3'] <= best_combo['volume_mean_3'][1]
                   and x['volume_mean_5'] >= best_combo['volume_mean_5'][0]
                   and x['volume_mean_5'] <= best_combo['volume_mean_5'][1]
                   and x['bullish_1'] >= best_combo['bullish_1'][0]
                   and x['bullish_1'] <= best_combo['bullish_1'][1]
                   and x['move_from_bottom'] >= best_combo['move_from_bottom'][0]
                   and x['move_from_bottom'] <= best_combo['move_from_bottom'][1])
    else 0, axis=1)

Average trades per day: 0.87
Success rate: 28.95%
Average profit per trade: $2,399.72


I found a more profitable combination that maintains a good number of trades while achieving better overall results. This combination:

1. Makes $3,398,000 in total profit
2. Generates about 1 trade per day (0.87)
3. Has a success rate of ~29%
4. Averages $2,399.72 profit per trade
5. Total trades: 1,416 (good sample size)

The key differences from the previous settings:
- Wider volume range (450-700)
- Higher volume_mean_3 requirement (600-700)
- More moderate volume_mean_5 range (500-700)
- Stricter bullish_1 threshold (0.52-0.60)
- Higher move_from_bottom minimum (0.12-0.20)

This combination provides a better balance between profitability and trading frequency while maintaining reasonable parameter ranges.

> Also build similar patters for bearish indicator.? the values for the profit  & loss remains same.

I'll create a similar analysis for the bearish indicator, using the same profit ($50,000) and loss ($17,000) values, focusing on identifying optimal parameter combinations.

In [451]:
# Let's create a function to calculate profit for bearish trades
def calculate_bearish_profit(group):
    profit = (group[group['y_true'] == 2].shape[0] * 50000) - \
             ((group[group['y_true'] != 2].shape[0]) * 17000)
    trades = len(group)
    return profit, trades

# Create ranges to test for each feature
feature_ranges = {
    'volume': [(450, 700), (500, 700), (550, 700), (600, 700)],
    'volume_mean_3': [(400, 750), (450, 750), (500, 750), (550, 750)],
    'volume_mean_5': [(450, 750), (500, 750), (550, 750), (600, 750)],
    'bearish_2': [(0.50, 0.65), (0.51, 0.65), (0.52, 0.65)],
    'move_from_top': [(0.08, 0.25), (0.09, 0.25), (0.10, 0.25)]
}

best_profit = float('-inf')
best_combo = None
best_trades = 0

# Test different combinations
for vol_range in feature_ranges['volume']:
    for vol3_range in feature_ranges['volume_mean_3']:
        for vol5_range in feature_ranges['volume_mean_5']:
            for bear_range in feature_ranges['bearish_2']:
                for move_range in feature_ranges['move_from_top']:
                    
                    # Apply conditions
                    filtered_data = base_data[
                        (base_data['volume'] >= vol_range[0]) &
                        (base_data['volume'] <= vol_range[1]) &
                        (base_data['volume_mean_3'] >= vol3_range[0]) &
                        (base_data['volume_mean_3'] <= vol3_range[1]) &
                        (base_data['volume_mean_5'] >= vol5_range[0]) &
                        (base_data['volume_mean_5'] <= vol5_range[1]) &
                        (base_data['bearish_2'] >= bear_range[0]) &
                        (base_data['bearish_2'] <= bear_range[1]) &
                        (base_data['move_from_top'] >= move_range[0]) &
                        (base_data['move_from_top'] <= move_range[1])
                    ]
                    
                    if len(filtered_data) > 0:
                        profit, trades = calculate_bearish_profit(filtered_data)
                        
                        # Update best combination if profit is higher and trades are reasonable
                        if profit > best_profit and trades >= 100:  # minimum 100 trades
                            best_profit = profit
                            best_combo = {
                                'volume': vol_range,
                                'volume_mean_3': vol3_range,
                                'volume_mean_5': vol5_range,
                                'bearish_2': bear_range,
                                'move_from_top': move_range
                            }
                            best_trades = trades

print("Best Bearish Combination Found:")
print(f"Profit: ${best_profit:,}")
print(f"Number of Trades: {best_trades}")
print("\nParameters:")
for feature, range_val in best_combo.items():
    print(f"{feature}: {range_val}")

# Let's analyze the distribution of y_true for this combination
best_filtered = base_data[
    (base_data['volume'] >= best_combo['volume'][0]) &
    (base_data['volume'] <= best_combo['volume'][1]) &
    (base_data['volume_mean_3'] >= best_combo['volume_mean_3'][0]) &
    (base_data['volume_mean_3'] <= best_combo['volume_mean_3'][1]) &
    (base_data['volume_mean_5'] >= best_combo['volume_mean_5'][0]) &
    (base_data['volume_mean_5'] <= best_combo['volume_mean_5'][1]) &
    (base_data['bearish_2'] >= best_combo['bearish_2'][0]) &
    (base_data['bearish_2'] <= best_combo['bearish_2'][1]) &
    (base_data['move_from_top'] >= best_combo['move_from_top'][0]) &
    (base_data['move_from_top'] <= best_combo['move_from_top'][1])
]

print("\nDistribution of outcomes:")
print(best_filtered['y_true'].value_counts())

# Calculate additional metrics
trades_per_day = best_trades / base_data['date'].nunique()
success_rate = (best_filtered[best_filtered['y_true'] == 2].shape[0] / best_trades) * 100

print(f"\nAverage trades per day: {trades_per_day:.2f}")
print(f"Success rate: {success_rate:.2f}%")
print(f"Average profit per trade: ${best_profit/best_trades:,.2f}")

# Update the bearish flag with the best combination
base_data['bearish_flag'] = base_data.apply(
    lambda x: 2 if (x['volume'] >= best_combo['volume'][0]
                   and x['volume'] <= best_combo['volume'][1]
                   and x['volume_mean_3'] >= best_combo['volume_mean_3'][0]
                   and x['volume_mean_3'] <= best_combo['volume_mean_3'][1]
                   and x['volume_mean_5'] >= best_combo['volume_mean_5'][0]
                   and x['volume_mean_5'] <= best_combo['volume_mean_5'][1]
                   and x['bearish_2'] >= best_combo['bearish_2'][0]
                   and x['bearish_2'] <= best_combo['bearish_2'][1]
                   and x['move_from_top'] >= best_combo['move_from_top'][0]
                   and x['move_from_top'] <= best_combo['move_from_top'][1])
    else 0, axis=1)

Best Bearish Combination Found:
Profit: $41,470,000
Number of Trades: 13948

Parameters:
volume: (450, 700)
volume_mean_3: (500, 750)
volume_mean_5: (450, 750)
bearish_2: (0.5, 0.65)
move_from_top: (0.08, 0.25)

Distribution of outcomes:
y_true
0    6003
2    4158
1    3377
3     410
Name: count, dtype: int64

Average trades per day: 8.53
Success rate: 29.81%
Average profit per trade: $2,973.19


I found a highly profitable combination for bearish trades that achieves excellent results. This combination:

1. Makes $41,470,000 in total profit
2. Generates about 9 trades per day (8.53)
3. Has a success rate of ~30%
4. Averages $2,973.19 profit per trade
5. Total trades: 13,948 (very good sample size)

The optimal parameters are:
- Volume: 450-700
- Volume Mean 3: 500-750
- Volume Mean 5: 450-750
- Bearish_2: 0.50-0.65
- Move from Top: 0.08-0.25

This combination provides significantly higher profitability than the bullish strategy, with more frequent trading opportunities while maintaining a similar success rate. The wider ranges for volume metrics and move_from_top allow for capturing more trading opportunities while still maintaining profitable selectivity.

> Create bullish flag find a combination which has highest value profit booked from the features mentioned below.
Every correct bullish prediction, earns a USD 5000 profit and non-bullish move losses USD1700. Can you create a combination which has both higher profitability and good amount of trades also being made (not much tighter restrictions are preferred).
use the features from the below:
   ,'move_from_bottom'
    ,'EMA_slope'
    ,'volume'
    ,'bearish_2'
    ,'bullish_1'
    ,'EMA_slope_60'
    ,'move_from_top'
    ,'volume_std_mean'
    ,'volume_mean_5'
    ,'volume_mean_3'
    ,'EMA_slope_60'
    ,'move_from_top'
    ,'volume_std_mean'
    ,'volume_mean_5'
    ,'volume_mean_3'

I'll help analyze these features to find the optimal combination for bullish predictions. I'll create a comprehensive search through feature combinations, with a focus on profitability while maintaining a reasonable number of trades.

In [7]:
# Let's create a function to calculate profit for bullish trades
def calculate_bullish_profit(group):
    profit = (group[group['y_true'] == 1].shape[0] * 5000) - \
             ((group[group['y_true'] != 1].shape[0]) * 1700)
    trades = len(group)
    return profit, trades

# Create ranges to test for each feature
feature_ranges = {
    'volume': [(450, 700), (500, 700), (550, 700)],
    'volume_mean_3': [(450, 700), (500, 700), (550, 700)],
    'volume_mean_5': [(450, 700), (500, 700), (550, 700)],
    'bullish_1': [(0.50, 0.60), (0.52, 0.60), (0.54, 0.60)],
    'move_from_bottom': [(0.10, 0.20), (0.12, 0.20), (0.14, 0.20)],
    'EMA_slope': [(0, float('inf')), (0.1, float('inf'))],
    'EMA_slope_60': [(-float('inf'), 0), (-float('inf'), -0.1)],
    'volume_std_mean': [(50, 150), (75, 150), (100, 150)]
}

best_profit = float('-inf')
best_combo = None
best_trades = 0

# Test combinations of core features first
for vol_range in feature_ranges['volume']:
    for vol3_range in feature_ranges['volume_mean_3']:
        for vol5_range in feature_ranges['volume_mean_5']:
            for bull_range in feature_ranges['bullish_1']:
                for move_range in feature_ranges['move_from_bottom']:
                    for ema_range in feature_ranges['EMA_slope']:
                        
                        # Apply conditions
                        filtered_data = base_data[
                            (base_data['volume'] >= vol_range[0]) &
                            (base_data['volume'] <= vol_range[1]) &
                            (base_data['volume_mean_3'] >= vol3_range[0]) &
                            (base_data['volume_mean_3'] <= vol3_range[1]) &
                            (base_data['volume_mean_5'] >= vol5_range[0]) &
                            (base_data['volume_mean_5'] <= vol5_range[1]) &
                            (base_data['bullish_1'] >= bull_range[0]) &
                            (base_data['bullish_1'] <= bull_range[1]) &
                            (base_data['move_from_bottom'] >= move_range[0]) &
                            (base_data['move_from_bottom'] <= move_range[1]) &
                            (base_data['EMA_slope'] >= ema_range[0]) &
                            (base_data['EMA_slope'] <= ema_range[1])
                        ]
                        
                        if len(filtered_data) > 0:
                            profit, trades = calculate_bullish_profit(filtered_data)
                            
                            # Update best combination if profit is higher and trades are reasonable
                            if profit > best_profit and trades >= 100:  # minimum 100 trades
                                best_profit = profit
                                best_combo = {
                                    'volume': vol_range,
                                    'volume_mean_3': vol3_range,
                                    'volume_mean_5': vol5_range,
                                    'bullish_1': bull_range,
                                    'move_from_bottom': move_range,
                                    'EMA_slope': ema_range
                                }
                                best_trades = trades

print("Best Bullish Combination Found:")
print(f"Profit: ${best_profit:,}")
print(f"Number of Trades: {best_trades}")
print("\nParameters:")
for feature, range_val in best_combo.items():
    print(f"{feature}: {range_val}")

# Analyze the distribution for the best combination
best_filtered = base_data[
    (base_data['volume'] >= best_combo['volume'][0]) &
    (base_data['volume'] <= best_combo['volume'][1]) &
    (base_data['volume_mean_3'] >= best_combo['volume_mean_3'][0]) &
    (base_data['volume_mean_3'] <= best_combo['volume_mean_3'][1]) &
    (base_data['volume_mean_5'] >= best_combo['volume_mean_5'][0]) &
    (base_data['volume_mean_5'] <= best_combo['volume_mean_5'][1]) &
    (base_data['bullish_1'] >= best_combo['bullish_1'][0]) &
    (base_data['bullish_1'] <= best_combo['bullish_1'][1]) &
    (base_data['move_from_bottom'] >= best_combo['move_from_bottom'][0]) &
    (base_data['move_from_bottom'] <= best_combo['move_from_bottom'][1]) &
    (base_data['EMA_slope'] >= best_combo['EMA_slope'][0]) &
    (base_data['EMA_slope'] <= best_combo['EMA_slope'][1])
]

print("\nDistribution of outcomes:")
print(best_filtered['y_true'].value_counts())

# Calculate additional metrics
trades_per_day = best_trades / base_data['date'].nunique()
success_rate = (best_filtered[best_filtered['y_true'] == 1].shape[0] / best_trades) * 100

print(f"\nAverage trades per day: {trades_per_day:.2f}")
print(f"Success rate: {success_rate:.2f}%")
print(f"Average profit per trade: ${best_profit/best_trades:,.2f}")

Best Bullish Combination Found:
Profit: $198,300
Number of Trades: 1247

Parameters:
volume: (550, 700)
volume_mean_3: (450, 700)
volume_mean_5: (450, 700)
bullish_1: (0.54, 0.6)
move_from_bottom: (0.1, 0.2)
EMA_slope: (0, inf)

Distribution of outcomes:
y_true
0    479
2    382
1    346
3     40
Name: count, dtype: int64


KeyError: 'date'

In [9]:
# Calculate additional metrics using datetime instead of date
trades_per_day = best_trades / base_data['datetime'].dt.date.nunique()
success_rate = (best_filtered[best_filtered['y_true'] == 1].shape[0] / best_trades) * 100

print(f"Average trades per day: {trades_per_day:.2f}")
print(f"Success rate: {success_rate:.2f}%")
print(f"Average profit per trade: ${best_profit/best_trades:,.2f}")

# Update the bullish flag with the best combination
base_data['bullish_flag_new'] = base_data.apply(
    lambda x: 1 if (x['volume'] >= best_combo['volume'][0]
                   and x['volume'] <= best_combo['volume'][1]
                   and x['volume_mean_3'] >= best_combo['volume_mean_3'][0]
                   and x['volume_mean_3'] <= best_combo['volume_mean_3'][1]
                   and x['volume_mean_5'] >= best_combo['volume_mean_5'][0]
                   and x['volume_mean_5'] <= best_combo['volume_mean_5'][1]
                   and x['bullish_1'] >= best_combo['bullish_1'][0]
                   and x['bullish_1'] <= best_combo['bullish_1'][1]
                   and x['move_from_bottom'] >= best_combo['move_from_bottom'][0]
                   and x['move_from_bottom'] <= best_combo['move_from_bottom'][1]
                   and x['EMA_slope'] >= best_combo['EMA_slope'][0]
                   and x['EMA_slope'] <= best_combo['EMA_slope'][1])
    else 0, axis=1)

Average trades per day: 0.76
Success rate: 27.75%
Average profit per trade: $159.02


I found an optimized combination for bullish trades that provides consistent profitability while maintaining a reasonable number of trading opportunities. Here's a summary of the results:

1. Total Profit: $198,300
2. Number of Trades: 1,247
3. Average trades per day: 0.76
4. Success rate: 27.75%
5. Average profit per trade: $159.02

The optimal parameters are:
- Volume: 550-700
- Volume Mean 3: 450-700
- Volume Mean 5: 450-700
- Bullish_1: 0.54-0.60
- Move from Bottom: 0.10-0.20
- EMA_slope: > 0 (positive slope)

This combination provides a balanced approach with:
- Higher volume threshold (550) for primary volume but more flexible range for moving averages
- Stricter bullish_1 signal requirement (0.54 minimum)
- Reasonable move_from_bottom range
- Positive EMA slope requirement for trend confirmation

The strategy generates about 1 trade per day with a reasonable success rate of nearly 28%. While the profit per trade is modest, the consistency and frequency of trades make it a viable strategy. The distribution of outcomes shows good selectivity in avoiding the worst outcomes (y_true = 3) while maintaining a balanced mix of other outcomes.

> create a balanced combination for the bullish flag with maximum possible profitability? Other things can be ignored.

I'll create a comprehensive search focusing solely on maximizing profitability for the bullish flag, without constraints on number of trades or other metrics.

In [37]:
# Let's create a function to calculate profit for bullish trades
def calculate_bullish_profit(group):
    profit = (group[group['y_true'] == 1].shape[0] * 5000) - \
             ((group[group['y_true'] != 1].shape[0]) * 1700)
    return profit

# Create ranges to test for each feature
feature_ranges = {
    'volume': [(400, 800), (450, 750), (500, 700), (550, 650)],
    'volume_mean_3': [(400, 800), (450, 750), (500, 700), (550, 650)],
    'volume_mean_5': [(400, 800), (450, 750), (500, 700), (550, 650)],
    'bullish_1': [(0.50, 0.65), (0.52, 0.62), (0.54, 0.60), (0.56, 0.58)],
    'move_from_bottom': [(0.05, 0.25), (0.08, 0.22), (0.10, 0.20), (0.12, 0.18)],
    'EMA_slope': [(-0.1, float('inf')), (0, float('inf')), (0.1, float('inf'))],
    'EMA_slope_60': [(-float('inf'), 0.1), (-float('inf'), 0), (-float('inf'), -0.1)]
}

best_profit = float('-inf')
best_combo = None
best_filtered = None

# Test different combinations
for vol_range in feature_ranges['volume']:
    for vol3_range in feature_ranges['volume_mean_3']:
        for vol5_range in feature_ranges['volume_mean_5']:
            for bull_range in feature_ranges['bullish_1']:
                for move_range in feature_ranges['move_from_bottom']:
                    for ema_range in feature_ranges['EMA_slope']:
                        for ema60_range in feature_ranges['EMA_slope_60']:
                            
                            # Apply conditions
                            filtered_data = base_data[
                                (base_data['volume'] >= vol_range[0]) &
                                (base_data['volume'] <= vol_range[1]) &
                                (base_data['volume_mean_3'] >= vol3_range[0]) &
                                (base_data['volume_mean_3'] <= vol3_range[1]) &
                                (base_data['volume_mean_5'] >= vol5_range[0]) &
                                (base_data['volume_mean_5'] <= vol5_range[1]) &
                                (base_data['bullish_1'] >= bull_range[0]) &
                                (base_data['bullish_1'] <= bull_range[1]) &
                                (base_data['move_from_bottom'] >= move_range[0]) &
                                (base_data['move_from_bottom'] <= move_range[1]) &
                                (base_data['EMA_slope'] >= ema_range[0]) &
                                (base_data['EMA_slope'] <= ema_range[1]) &
                                (base_data['EMA_slope_60'] >= ema60_range[0]) &
                                (base_data['EMA_slope_60'] <= ema60_range[1])
                            ]
                            
                            if len(filtered_data) > 0:
                                profit = calculate_bullish_profit(filtered_data)
                                
                                if profit > best_profit:
                                    best_profit = profit
                                    best_combo = {
                                        'volume': vol_range,
                                        'volume_mean_3': vol3_range,
                                        'volume_mean_5': vol5_range,
                                        'bullish_1': bull_range,
                                        'move_from_bottom': move_range,
                                        'EMA_slope': ema_range,
                                        'EMA_slope_60': ema60_range
                                    }
                                    best_filtered = filtered_data

print("Best Bullish Combination Found:")
print(f"Total Profit: ${best_profit:,}")
print("\nParameters:")
for feature, range_val in best_combo.items():
    print(f"{feature}: {range_val}")

print("\nDistribution of outcomes:")
print(best_filtered['y_true'].value_counts())

# Calculate metrics
trades = len(best_filtered)
trades_per_day = trades / base_data['datetime'].dt.date.nunique()
success_rate = (best_filtered[best_filtered['y_true'] == 1].shape[0] / trades) * 100
profit_per_trade = best_profit/trades

print(f"\nNumber of trades: {trades}")
print(f"Average trades per day: {trades_per_day:.2f}")
print(f"Success rate: {success_rate:.2f}%")
print(f"Average profit per trade: ${profit_per_trade:.2f}")

Best Bullish Combination Found:
Total Profit: $776,500

Parameters:
volume: (500, 700)
volume_mean_3: (400, 800)
volume_mean_5: (400, 800)
bullish_1: (0.5, 0.65)
move_from_bottom: (0.08, 0.22)
EMA_slope: (-0.1, inf)
EMA_slope_60: (-inf, -0.1)

Distribution of outcomes:
y_true
0    1140
1     837
2     804
3      61
Name: count, dtype: int64

Number of trades: 2842
Average trades per day: 1.74
Success rate: 29.45%
Average profit per trade: $273.22


In [39]:
# Update the bullish flag with this most profitable combination
base_data['bullish_flag_new'] = base_data.apply(
    lambda x: 1 if (x['volume'] >= best_combo['volume'][0]
                   and x['volume'] <= best_combo['volume'][1]
                   and x['volume_mean_3'] >= best_combo['volume_mean_3'][0]
                   and x['volume_mean_3'] <= best_combo['volume_mean_3'][1]
                   and x['volume_mean_5'] >= best_combo['volume_mean_5'][0]
                   and x['volume_mean_5'] <= best_combo['volume_mean_5'][1]
                   and x['bullish_1'] >= best_combo['bullish_1'][0]
                   and x['bullish_1'] <= best_combo['bullish_1'][1]
                   and x['move_from_bottom'] >= best_combo['move_from_bottom'][0]
                   and x['move_from_bottom'] <= best_combo['move_from_bottom'][1]
                   and x['EMA_slope'] >= best_combo['EMA_slope'][0]
                   and x['EMA_slope'] <= best_combo['EMA_slope'][1]
                   and x['EMA_slope_60'] >= best_combo['EMA_slope_60'][0]
                   and x['EMA_slope_60'] <= best_combo['EMA_slope_60'][1])
    else 0, axis=1)

I found a significantly more profitable combination for bullish trades by optimizing purely for profitability. The new combination achieves:

1. Total Profit: $776,500 (nearly 4x improvement)
2. Number of Trades: 2,842
3. Average trades per day: 1.74
4. Success rate: 29.45%
5. Average profit per trade: $273.22

Key features of this optimal combination:
- More selective volume range (500-700) but wider ranges for moving averages (400-800)
- Broader bullish_1 signal range (0.50-0.65)
- Moderate move_from_bottom range (0.08-0.22)
- Allows slightly negative EMA_slope (-0.1 to inf)
- Requires negative long-term trend (EMA_slope_60 < -0.1)

This combination captures more trading opportunities while maintaining a good success rate and significantly higher profitability per trade. The negative long-term trend requirement combined with flexible short-term slope appears to identify better entry points for bullish moves.

> can you check how many bullish_1 > 0.7 were profitable. Give how many trades were made, avg trade per day and profitability. 

I'll analyze the profitability of trades where bullish_1 is greater than 0.7.

In [63]:
# Filter data where bullish_1 > 0.7
high_bullish = base_data[base_data['bullish_1'] > 0.75]

# Calculate metrics
total_trades = len(high_bullish)
profitable_trades = len(high_bullish[high_bullish['y_true'] == 1])
days = base_data['datetime'].dt.date.nunique()
trades_per_day = total_trades / days

# Calculate profitability (5000 for correct bullish, -1700 for incorrect)
profit = (profitable_trades * 5000) - ((total_trades - profitable_trades) * 1700)

print(f"Total trades: {total_trades}")
print(f"Profitable trades: {profitable_trades}")
print(f"Average trades per day: {trades_per_day:.2f}")
print(f"Total profit: ${profit:,}")
print(f"Profit per trade: ${profit/total_trades:.2f}")
print(f"Success rate: {(profitable_trades/total_trades)*100:.2f}%")

# Show distribution of outcomes
print("\nOutcome distribution:")
print(high_bullish['y_true'].value_counts())

Total trades: 24704
Profitable trades: 7346
Average trades per day: 15.10
Total profit: $7,221,400
Profit per trade: $292.32
Success rate: 29.74%

Outcome distribution:
y_true
0    7642
2    7364
1    7346
3    2352
Name: count, dtype: int64


> can you find a combination for all the bullish_1> 0.7 compared to the bullish movement vs non-bullish movement. See if volume, ema slope values has help distinguish the movement somehow. 

I'll analyze the high bullish signals (bullish_1 > 0.7) and look for patterns in volume and EMA slopes that distinguish between successful and unsuccessful bullish movements.

In [65]:
# Analyze high bullish signals with various features
high_bullish = base_data[base_data['bullish_1'] > 0.7].copy()

# Create success flag (1 for bullish movement, 0 for others)
high_bullish['success'] = (high_bullish['y_true'] == 1).astype(int)

# Calculate mean values for key metrics grouped by success
analysis = high_bullish.groupby('success').agg({
    'volume': ['mean', 'std'],
    'volume_mean_3': ['mean', 'std'],
    'volume_mean_5': ['mean', 'std'],
    'EMA_slope': ['mean', 'std'],
    'EMA_slope_15': ['mean', 'std'],
    'EMA_slope_60': ['mean', 'std'],
    'move_from_bottom': ['mean', 'std']
}).round(3)

print("Feature Analysis for Bullish_1 > 0.7:")
print("\nMean and Standard Deviation by Success (1) vs Failure (0):")
print(analysis)

# Calculate success rate by volume ranges
volume_ranges = [(0, 300), (300, 500), (500, 700), (700, 1000), (1000, float('inf'))]
volume_analysis = []

for low, high in volume_ranges:
    mask = (high_bullish['volume'] >= low) & (high_bullish['volume'] < high)
    subset = high_bullish[mask]
    if len(subset) > 0:
        success_rate = (subset['success'].mean() * 100)
        trades = len(subset)
        volume_analysis.append({
            'Volume Range': f"{low}-{high}",
            'Success Rate %': round(success_rate, 2),
            'Trades': trades
        })

print("\nSuccess Rate by Volume Range:")
for result in volume_analysis:
    print(f"Volume {result['Volume Range']}: {result['Success Rate']}% ({result['Trades']} trades)")

Feature Analysis for Bullish_1 > 0.7:

Mean and Standard Deviation by Success (1) vs Failure (0):
          volume          volume_mean_3          volume_mean_5           \
            mean      std          mean      std          mean      std   
success                                                                   
0        784.644  646.332       803.974  639.433       815.678  622.617   
1        893.383  624.690       914.111  631.588       918.342  604.766   

        EMA_slope        EMA_slope_15        EMA_slope_60         \
             mean    std         mean    std         mean    std   
success                                                            
0           0.014  2.644        0.011  2.045        0.010  0.986   
1           0.025  2.659        0.029  2.101        0.008  1.070   

        move_from_bottom         
                    mean    std  
success                          
0                  0.375  0.319  
1                  0.406  0.330  

Success Rate b

KeyError: 'Success Rate'

In [67]:
# Let's fix the volume analysis print statement
print("\nSuccess Rate by Volume Range:")
for result in volume_analysis:
    print(f"Volume {result['Volume Range']}: {result['Success Rate %']}% ({result['Trades']} trades)")

# Let's also analyze EMA slope ranges
ema_ranges = [
    (-float('inf'), -0.5),
    (-0.5, 0),
    (0, 0.5),
    (0.5, float('inf'))
]

ema_analysis = []
for low, high in ema_ranges:
    mask = (high_bullish['EMA_slope'] >= low) & (high_bullish['EMA_slope'] < high)
    subset = high_bullish[mask]
    if len(subset) > 0:
        success_rate = (subset['success'].mean() * 100)
        trades = len(subset)
        ema_analysis.append({
            'EMA Range': f"{low:.1f} to {high:.1f}",
            'Success Rate': round(success_rate, 2),
            'Trades': trades
        })

print("\nSuccess Rate by EMA Slope Range:")
for result in ema_analysis:
    print(f"EMA Slope {result['EMA Range']}: {result['Success Rate']}% ({result['Trades']} trades)")


Success Rate by Volume Range:
Volume 0-300: 17.68% (4226 trades)
Volume 300-500: 27.48% (4629 trades)
Volume 500-700: 30.93% (4410 trades)
Volume 700-1000: 33.74% (4481 trades)
Volume 1000-inf: 35.23% (6958 trades)

Success Rate by EMA Slope Range:
EMA Slope -inf to -0.5: 31.57% (9163 trades)
EMA Slope -0.5 to 0.0: 25.79% (3315 trades)
EMA Slope 0.0 to 0.5: 24.99% (3269 trades)
EMA Slope 0.5 to inf: 31.05% (8957 trades)


In [69]:
# Let's analyze combinations of volume and EMA slope
# We'll focus on the most promising ranges

# Define the most promising ranges based on previous analysis
volume_ranges = [(500, 700), (700, 1000), (1000, float('inf'))]
ema_ranges = [(-float('inf'), -0.5), (0.5, float('inf'))]

print("Analysis of Volume + EMA Slope Combinations:")
print("==========================================")

for vol_low, vol_high in volume_ranges:
    for ema_low, ema_high in ema_ranges:
        mask = (
            (high_bullish['volume'] >= vol_low) & 
            (high_bullish['volume'] < vol_high) &
            (high_bullish['EMA_slope'] >= ema_low) & 
            (high_bullish['EMA_slope'] < ema_high)
        )
        subset = high_bullish[mask]
        if len(subset) > 0:
            success_rate = (subset['success'].mean() * 100)
            trades = len(subset)
            profit = (subset['success'].sum() * 5000) - ((len(subset) - subset['success'].sum()) * 1700)
            profit_per_trade = profit / trades if trades > 0 else 0
            
            print(f"\nVolume {vol_low}-{vol_high}, EMA Slope {ema_low:.1f} to {ema_high:.1f}")
            print(f"Success Rate: {success_rate:.2f}%")
            print(f"Number of Trades: {trades}")
            print(f"Total Profit: ${profit:,.2f}")
            print(f"Profit per Trade: ${profit_per_trade:.2f}")
            print(f"Trades per Day: {trades/days:.2f}")

Analysis of Volume + EMA Slope Combinations:

Volume 500-700, EMA Slope -inf to -0.5
Success Rate: 30.47%
Number of Trades: 1582
Total Profit: $540,000.00
Profit per Trade: $341.34
Trades per Day: 0.97

Volume 500-700, EMA Slope 0.5 to inf
Success Rate: 32.32%
Number of Trades: 1612
Total Profit: $750,300.00
Profit per Trade: $465.45
Trades per Day: 0.99

Volume 700-1000, EMA Slope -inf to -0.5
Success Rate: 33.60%
Number of Trades: 1780
Total Profit: $980,600.00
Profit per Trade: $550.90
Trades per Day: 1.09

Volume 700-1000, EMA Slope 0.5 to inf
Success Rate: 35.37%
Number of Trades: 1685
Total Profit: $1,128,700.00
Profit per Trade: $669.85
Trades per Day: 1.03

Volume 1000-inf, EMA Slope -inf to -0.5
Success Rate: 35.61%
Number of Trades: 3100
Total Profit: $2,126,800.00
Profit per Trade: $686.06
Trades per Day: 1.89

Volume 1000-inf, EMA Slope 0.5 to inf
Success Rate: 34.96%
Number of Trades: 3021
Total Profit: $1,939,500.00
Profit per Trade: $642.01
Trades per Day: 1.85


Based on the analysis of trades with bullish_1 > 0.7, several key patterns emerge:

1. Volume Impact:
- Higher volume correlates with better success rates
- Success rates increase steadily with volume:
  * 17.68% for volume 0-300
  * 35.23% for volume >1000

2. EMA Slope Patterns:
- Strong negative (<-0.5) and strong positive (>0.5) slopes perform best
- Success rates:
  * 31.57% for strong negative slope
  * 31.05% for strong positive slope
  * Lower success rates (24-25%) for moderate slopes

3. Best Combinations:
The most profitable combination is:
- Volume >1000 with strong negative EMA slope
  * 35.61% success rate
  * $686.06 profit per trade
  * 1.89 trades per day
  * Total profit: $2,126,800

Second best:
- Volume 700-1000 with positive EMA slope
  * 35.37% success rate
  * $669.85 profit per trade
  * 1.03 trades per day
  * Total profit: $1,128,700

This suggests that high volume combined with strong trend movement (either direction) provides the best trading opportunities when bullish_1 > 0.7.

In [81]:
import pandas as pd
history = pd.read_csv('/work/data/processed/transformed_data.csv')
updated_preds = pd.read_csv('/work/data/predictions/predictions_data.csv')
pred_actual = pd.read_csv('/work/data/predictions/predictions_actuals.csv')

In [82]:
history = pd.merge(
    history,
    updated_preds,
    on='datetime',
    how='inner'
)
history = pd.merge(
    history,
    pred_actual,
    on='datetime',
    how='inner'
)

In [75]:
import json

# Open and read the file first, then parse the JSON
with open('/work/config/strategy_volume_based.json', 'r') as file:
    config = json.load(file)  # Use json.load() for files instead of json.loads()

In [88]:
history[(history['volume']> 600)
        & (history['bullish_1']> .54)
        #& ((history['EMA_slope_15']>config['ema_slope_15_threshold_max']) | (history['EMA_slope_15']<config['ema_slope_15_threshold_min']))
][['datetime','y_actual','EMA_slope_15','EMA_slope','bullish_1','volume','move_from_bottom']]

,datetime,y_actual,EMA_slope_15,EMA_slope,bullish_1,volume,move_from_bottom
60,2025-04-03 11:20:00+00:00,1,-2.091650,-3.568183,0.591361,1392,0.776905
61,2025-04-03 11:25:00+00:00,1,-0.805193,-1.214547,0.580584,1197,0.821725
62,2025-04-03 11:30:00+00:00,1,-0.092044,0.008363,0.559202,1857,0.893566
63,2025-04-03 11:35:00+00:00,1,2.581961,4.266690,0.659798,1788,0.893179
64,2025-04-03 11:40:00+00:00,1,2.396716,3.633352,0.586857,1767,0.893928
...,...,...,...,...,...,...,...
1010,2025-04-08 18:35:00+00:00,1,2.004276,2.329996,0.575777,1088,0.544793
1011,2025-04-08 18:40:00+00:00,1,1.378741,1.263997,0.571937,992,0.525730
1019,2025-04-08 19:20:00+00:00,2,2.853435,3.609113,0.578610,1204,0.721456
1021,2025-04-08 19:30:00+00:00,2,1.264348,0.949832,0.543622,1026,0.601586


In [90]:
history[(history['volume']> 600)
        & (history['bearish_2']> .54)
        & ((history['EMA_slope_15']>config['ema_slope_15_threshold_max']) | (history['EMA_slope_15']<config['ema_slope_15_threshold_min']))
][['datetime','y_actual','EMA_slope_15','EMA_slope','bullish_1','volume','move_from_bottom']].iloc[:-15]
#][['y_actual']].value_counts()

,datetime,y_actual,EMA_slope_15,EMA_slope,bullish_1,volume,move_from_bottom
60,2025-04-03 11:20:00+00:00,1,-2.091650,-3.568183,0.591361,1392,0.776905
61,2025-04-03 11:25:00+00:00,1,-0.805193,-1.214547,0.580584,1197,0.821725
63,2025-04-03 11:35:00+00:00,1,2.581961,4.266690,0.659798,1788,0.893179
64,2025-04-03 11:40:00+00:00,1,2.396716,3.633352,0.586857,1767,0.893928
65,2025-04-03 11:45:00+00:00,3,2.672127,3.826682,0.574543,1748,1.009118
...,...,...,...,...,...,...,...
917,2025-04-08 10:50:00+00:00,1,0.346628,0.323095,0.555165,1393,0.210349
922,2025-04-08 11:15:00+00:00,3,1.239031,1.795184,0.443977,1614,0.291508
926,2025-04-08 11:35:00+00:00,1,-0.396286,-1.066613,0.445693,1510,0.185702
970,2025-04-08 15:15:00+00:00,1,-1.821038,-2.078132,0.518955,1592,0.132077


In [1]:
base_data[(base_data['volume']> 900)
        & (base_data['bullish_1']> 0.72)
        & ((base_data['EMA_slope_15']>0.25) | (base_data['EMA_slope_15']<-0.25))
        #& (base_data['move_from_bottom']>0.15) #& (base_data['move_from_bottom']<0.25) 
]['y_true'].value_counts()

NameError: name 'base_data' is not defined

In [355]:
(predicted[1] * 3000) - ((predicted[0]+predicted[2]+predicted[3])*1000)

951000

In [63]:
history[(history['bearish_2']> 0.7)
        & (history['volume']> 600)       
        & ((history['EMA_slope']>0.25) | (history['EMA_slope']<-0.25))
        #& (history['move_from_top']<0.45) & (history['move_from_top']>0.12)
][['datetime','y_true','EMA_slope_15','EMA_slope','bullish_1','volume','move_from_bottom']]

,datetime,y_true,EMA_slope_15,EMA_slope,bullish_1,volume,move_from_bottom
63,2025-04-03 11:35:00+00:00,1,2.581961,4.266690,0.659798,1788,0.893179
98,2025-04-03 14:30:00+00:00,3,0.188790,0.306724,0.682424,1778,0.577140


In [41]:
base_data[(base_data['volume']> 900)
        & (base_data['bearish_2']> 0.72)
        & ((base_data['EMA_slope_15']>0.25) | (base_data['EMA_slope_15']<-0.25))
        #& (base_data['move_from_top']<0.35) & (base_data['move_from_top']>0.12)
]['y_true'].value_counts()
#predicted

y_true
2    1784
1    1665
3    1044
0     423
Name: count, dtype: int64

In [47]:
history[(history['volume']> 700)
        & (history['bearish_2']> 0.7)
        & ((history['EMA_slope_15']>0.25) | (history['EMA_slope_15']<-0.25))
        & (history['move_from_top']<0.45) & (history['move_from_top']>0.15)
]['y_true'].value_counts()

Series([], Name: count, dtype: int64)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a127aa9e-0a77-4af9-a6ce-85e7a9b74042' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>